In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e3/sample_submission.csv
/kaggle/input/playground-series-s5e3/train.csv
/kaggle/input/playground-series-s5e3/test.csv
/kaggle/input/rainfall-prediction-using-machine-learning/Rainfall.csv


In [2]:
# !git clone https://github.com/PriorLabs/tabpfn-extensions
!pip install tabpfn
# !pip install -e tabpfn-extensions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 5.6 MB/s eta 0:00:00


In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s5e3/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e3/test.csv")
orig = pd.read_csv("/kaggle/input/rainfall-prediction-using-machine-learning/Rainfall.csv")
display(train.info(), train.head(), train.describe().T)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2190 entries, 0 to 2189
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2190 non-null   int64  
 1   day            2190 non-null   int64  
 2   pressure       2190 non-null   float64
 3   maxtemp        2190 non-null   float64
 4   temparature    2190 non-null   float64
 5   mintemp        2190 non-null   float64
 6   dewpoint       2190 non-null   float64
 7   humidity       2190 non-null   float64
 8   cloud          2190 non-null   float64
 9   sunshine       2190 non-null   float64
 10  winddirection  2190 non-null   float64
 11  windspeed      2190 non-null   float64
 12  rainfall       2190 non-null   int64  
dtypes: float64(10), int64(3)
memory usage: 222.5 KB


None

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,0,1,1017.4,21.2,20.6,19.9,19.4,87.0,88.0,1.1,60.0,17.2,1
1,1,2,1019.5,16.2,16.9,15.8,15.4,95.0,91.0,0.0,50.0,21.9,1
2,2,3,1024.1,19.4,16.1,14.6,9.3,75.0,47.0,8.3,70.0,18.1,1
3,3,4,1013.4,18.1,17.8,16.9,16.8,95.0,95.0,0.0,60.0,35.6,1
4,4,5,1021.8,21.3,18.4,15.2,9.6,52.0,45.0,3.6,40.0,24.8,0


,count,mean,std,min,25%,50%,75%,max
id,2190.0,1094.500000,632.342866,0.0,547.250,1094.50,1641.750,2189.0
day,2190.0,179.948402,105.203592,1.0,89.000,178.50,270.000,365.0
pressure,2190.0,1013.602146,5.655366,999.0,1008.600,1013.00,1017.775,1034.6
maxtemp,2190.0,26.365799,5.654330,10.4,21.300,27.80,31.200,36.0
temparature,2190.0,23.953059,5.222410,7.4,19.300,25.50,28.400,31.5
mintemp,2190.0,22.170091,5.059120,4.0,17.700,23.85,26.400,29.8
dewpoint,2190.0,20.454566,5.288406,-0.3,16.800,22.15,25.000,26.7
humidity,2190.0,82.036530,7.800654,39.0,77.000,82.00,88.000,98.0
cloud,2190.0,75.721918,18.026498,2.0,69.000,83.00,88.000,100.0
sunshine,2190.0,3.744429,3.626327,0.0,0.400,2.40,6.800,12.1


In [4]:
orig.columns = orig.columns.str.strip()
orig['rainfall'] = orig['rainfall'].str.lower().map({'yes': 1, 'no': 0})
train = train.drop(columns=['id'])
train = pd.concat([orig, train], axis=0, ignore_index=True)
train = train.fillna(train.mean())
test = test.fillna(test.mean())
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            2556 non-null   int64  
 1   pressure       2556 non-null   float64
 2   maxtemp        2556 non-null   float64
 3   temparature    2556 non-null   float64
 4   mintemp        2556 non-null   float64
 5   dewpoint       2556 non-null   float64
 6   humidity       2556 non-null   float64
 7   cloud          2556 non-null   float64
 8   rainfall       2556 non-null   int64  
 9   sunshine       2556 non-null   float64
 10  winddirection  2556 non-null   float64
 11  windspeed      2556 non-null   float64
dtypes: float64(10), int64(2)
memory usage: 239.8 KB


### Zero New Features to See How Model Performs:
Not expecting any crazy results, just curious to see how a model can perform with no engineering for new features or tuning :P

In [5]:
X = train.drop(columns=["rainfall"])
y = train["rainfall"]
display(X.info(), y.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2556 entries, 0 to 2555
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            2556 non-null   int64  
 1   pressure       2556 non-null   float64
 2   maxtemp        2556 non-null   float64
 3   temparature    2556 non-null   float64
 4   mintemp        2556 non-null   float64
 5   dewpoint       2556 non-null   float64
 6   humidity       2556 non-null   float64
 7   cloud          2556 non-null   float64
 8   sunshine       2556 non-null   float64
 9   winddirection  2556 non-null   float64
 10  windspeed      2556 non-null   float64
dtypes: float64(10), int64(1)
memory usage: 219.8 KB
<class 'pandas.core.series.Series'>
RangeIndex: 2556 entries, 0 to 2555
Series name: rainfall
Non-Null Count  Dtype
--------------  -----
2556 non-null   int64
dtypes: int64(1)
memory usage: 20.1 KB


None

None

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from tabpfn import TabPFNClassifier
# from sklearn.inspection import permutation_importance

# split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

# Fit TabPFN
# model = TabPFNClassifier(device = "cuda", random_state = 42)
# model.fit(X_train, y_train)

# Compute feature importance
# result = permutation_importance(model, X_val, y_val, scoring="roc_auc")

# Print importance scores
# for i, score in enumerate(result.importances_mean):
#    print(f"Feature Importance of {X_train.columns[i]}: {score}")

In [7]:
# help the skew/imbalance
smote = SMOTE(sampling_strategy=1.0, random_state=42)
X_train_resampled, y_train_resampled = X_train, y_train#smote.fit_resample(X_train, y_train)

# scale data, gave minor boost to MI scores
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_resampled), columns=X_train_resampled.columns)

In [8]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# find most relevant features
selector = SelectKBest(mutual_info_classif, k=11)
X_train_selected = selector.fit_transform(X_train_resampled, y_train_resampled)
selected_features = X_train_resampled.columns[selector.get_support()]
print("Selected Features:", selected_features.tolist())

X_train_selected = X_train_resampled[selected_features]
X_val_selected = X_val[selected_features]

Selected Features: ['day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']


In [9]:
display(X_train_selected.info(), y_train_resampled.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2044 entries, 877 to 2501
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            2044 non-null   int64  
 1   pressure       2044 non-null   float64
 2   maxtemp        2044 non-null   float64
 3   temparature    2044 non-null   float64
 4   mintemp        2044 non-null   float64
 5   dewpoint       2044 non-null   float64
 6   humidity       2044 non-null   float64
 7   cloud          2044 non-null   float64
 8   sunshine       2044 non-null   float64
 9   winddirection  2044 non-null   float64
 10  windspeed      2044 non-null   float64
dtypes: float64(10), int64(1)
memory usage: 191.6 KB
<class 'pandas.core.series.Series'>
Index: 2044 entries, 877 to 2501
Series name: rainfall
Non-Null Count  Dtype
--------------  -----
2044 non-null   int64
dtypes: int64(1)
memory usage: 31.9 KB


None

None

In [10]:
from sklearn.metrics import roc_auc_score
from tabpfn import TabPFNClassifier

# train
model = TabPFNClassifier(device = "cuda", random_state = 12)
model.fit(X_train_selected, y_train_resampled)

# predict
val_probs = model.predict_proba(X_val_selected)
print("ROC AUC: ", roc_auc_score(y_val, val_probs[:, 1]))
# ROC AUC with no feature selection, no scaling, no smote:  0.871420668580912
# ROC AUC with feature selection, scaling, and smote: 0.8644134769895417
# ROC AUC with autotabpfn, feature selection, scaling, and smote: 0.8608835383683253
# ROC AUC with autotabpfn and no smote: 0.8702879270830589

# Using Orig Dataset + Given Dataset:
# using smote and 8 features: 0.8537081339712919
# smote and all features: 0.852811004784689
# no smote and all features: 0.8685007974481659
# no smote and 8 features: 0.8673644338118023
# no smote, all features, stratify, random = 12: 0.9004186602870813
# no smote, 8 features, stratify: 0.8973086124401913

/usr/local/lib/python3.10/dist-packages/tabpfn/base.py:88: UserWarning: Downloading model to /root/.cache/tabpfn/tabpfn-v2-classifier.ckpt.
  model, _, config_ = load_model_criterion_config(


tabpfn-v2-classifier.ckpt:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

ROC AUC:  0.9004186602870813


In [11]:
# predict on test
test = test.drop(columns="id")
test_probs = model.predict_proba(test[selected_features])[:, 1]

In [12]:
# create submission
submission = pd.read_csv("/kaggle/input/playground-series-s5e3/sample_submission.csv")
submission['rainfall'] = test_probs
submission.to_csv('submission_final.csv', index=False)
submission.head()

,id,rainfall
0,2190,0.991547
1,2191,0.992303
2,2192,0.960819
3,2193,0.103656
4,2194,0.051200
